In [20]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

# Define data path
DATA_PATH = "data/"
CHECKPOINT_PATH = "models/"

In [21]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targs = pickle.load(open(os.path.join(DATA_PATH, 'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [22]:
class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        return(len(self.x))
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

In [23]:
# dataset = CustomDataset(seqs, targs)
dataset = CustomDataset(seqs, prob_targs)

In [24]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        x_masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        y: a tensor of shape (# patiens, max # categories) of type torch.long
        y_masks: a tensor of shape (# patiens, max # categories) of type torch.bool
    """
#     sequences, targets = zip(*data)

#     num_patients = len(sequences)
#     num_visits = [len(patient) for patient in sequences]
#     num_codes = [len(visit) for patient in sequences for visit in patient]
#     batch_num_categories = [len(visit) for patient in targets for visit in patient]

#     max_num_visits = max(num_visits)
#     max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    
#     x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
#     x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
#     y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient[:-1]):
#             for k_code, code in enumerate(visit):
#                 x[i_patient, j_visit, k_code] = code
#                 x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    
#     return x, x_masks, y, y_masks

    sequences, targets = zip(*data)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
#     max_num_categories = max(batch_num_categories)
    max_num_categories = len(targets[0][0])
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
#     y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.float)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1

#     for i_patient, patient in enumerate(targets):
# #         import pdb; pdb.set_trace()
#         last_visit = patient[-1]
#         y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
#         y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
    for i_patient, patient in enumerate(targets):
#         import pdb; pdb.set_trace()
        last_visit = patient[-1]
        y[i_patient] = torch.FloatTensor(last_visit)
        y_masks[i_patient] = torch.BoolTensor(np.ones(max_num_categories))
    
    return x, x_masks, y, y_masks

In [25]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [26]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [27]:
def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    '''
    Arguments:
        train_dataset: train dataset of type `CustomDataset`
        test_dataset: test dataset of type `CustomDataset`
        val_dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, test_loader, val_loader: train, test and validation dataloaders
    '''
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [28]:
def sum_embeddings_with_mask(x, masks):
    """
    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, 
            embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    x[~masks] = 0
    return x.sum(2)

In [29]:
def get_last_visit(hidden_states, masks):
    """
    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim)
    """
    idx_vector = masks.any(dim=2).sum(1) - 1
    p_idx = torch.arange(0,len(hidden_states), dtype=torch.int64)
    last_hidden_state = hidden_states[p_idx,idx_vector]
    return last_hidden_state

In [30]:
def attention_sum(alpha, v, masks):
    """
    Arguments:
        alpha: the alpha attention weights of shape (batch_size, # visits, 1)
        v: the visit embeddings of shape (batch_size, # visits, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        c: the context vector of shape (batch_size, embedding_dim)
    """
    masks = masks[:,:].any(dim=2)
    v_masked = v.clone()
    v_masked[~masks] = 0
    c = torch.sum(alpha * v_masked, dim=1)
    return c

In [31]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """
        Arguments:
            hidden_dim: the hidden dimension
        """
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        """
        Arguments:
            g: the output tensor from RNN-alpha of shape (batch_size, # visits, hidden_dim) 
        
        Outputs:
            alpha: the corresponding attention weights of shape (batch_size, # visits, 1)
        """
        m = nn.Softmax(dim=1)
        return m(self.a_att(g))

In [32]:
class BaselineRNNa(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
            num_categories: total number of higher level categories to predict
        """
        self.embedding = nn.Embedding(num_codes, embedding_dim=180)
        self.rnn = nn.GRU(180, hidden_size=128, batch_first=True)
        self.att_a = AlphaAttention(128)
        self.fc = nn.Linear(180, num_categories)
        
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            logits: logits of shape (batch_size, # categories)
        """
        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        alpha = self.att_a(output)
        c = attention_sum(alpha, x, masks)
        logits = self.fc(c)        
        return logits
    
# load the model here
baseline_rnna = BaselineRNNa(num_codes = len(codes), num_categories=len(sub_categories))
baseline_rnna

BaselineRNNa(
  (embedding): Embedding(4903, 180)
  (rnn): GRU(180, 128, batch_first=True)
  (att_a): AlphaAttention(
    (a_att): Linear(in_features=128, out_features=1, bias=True)
  )
  (fc): Linear(in_features=180, out_features=184, bias=True)
)

In [33]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_rnna.parameters(), weight_decay=0.001)

In [34]:
def eval_model(model, test_loader, k=15, n=-1):
    """
    Arguments:
        model: the BaselineRNNa model
        test_loader: validation dataloader
        k: value for top k predictions
        n: num of records to evaluate in the batch, value -1 evaulates all records
        
    Outputs:
        precision_k: visit-level precison@k
        accuracy_k: code-level accuracy@k
    """
#     y_pred = torch.LongTensor()
#     y_true = torch.LongTensor()
#     all_precision = []
#     all_accuracy = []
    
#     model.eval()
#     with torch.no_grad():
#         for x, masks, y, y_masks in test_loader:
#             n_eval = y.shape[0] - 1 if n == -1 else n
#             y_hat = model(x, masks)
#             y_hat = F.softmax(y_hat, dim=-1)
#             y_multihot = indices_to_multihot(y, y_masks, y_hat)
#             k_correct = 0
#             total_precision = 0
#             total_accuracy = 0
#             for i in range(n_eval):
#                 visit_correct = 0
#                 y_true = y[i, y_masks[i]]
#                 _, y_pred = torch.topk(y_hat[i], k)

#                 for v in y_true:
#                     if v in y_pred:
#                         visit_correct += 1
                        
#                 visit_precision = visit_correct / min(k, len(y_true))
#                 visit_accuracy = visit_correct / len(y_true)
#                 k_correct += visit_correct
#                 total_precision += visit_precision
#                 total_accuracy += visit_accuracy

#             precision_k = total_precision / n_eval
#             accuracy_k = total_accuracy / n_eval
#             all_precision.append(precision_k)
#             all_accuracy.append(accuracy_k)
            
#     total_precision_k = np.mean(all_precision)
#     total_accuracy_k = np.mean(all_accuracy)
#     return total_precision_k, total_accuracy_k

    y_pred = torch.LongTensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y, y_masks in test_loader:
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
            #y_multihot = indices_to_multihot(y, y_masks, y_hat)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                #y_true = y[i, y_masks[i]]
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [35]:
def train(model, train_loader, test_loader, n_epochs):
    """
    Arguments:
        model: the BaselineeRNNa model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
    """
#     max_cpu, max_ram = print_cpu_usage()
#     for epoch in range(n_epochs):
#         model.train()
#         train_loss = 0
#         for x, x_masks, y, y_masks in train_loader:
#             y_hat = model(x, x_masks)
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
#             loss = criterion(y_hat, y_mh)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item()
#         train_loss = train_loss / len(train_loader)
#         cpu, ram = print_cpu_usage()
#         max_cpu = cpu if cpu > max_cpu else max_cpu
#         max_ram = ram if ram > max_ram else max_ram
#         print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
#         for k in range(5, 31, 5):
#             precision_k, accuracy_k = eval_model(model, test_loader, k=k)
#             print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
#     final_cpu, final_ram = print_cpu_usage()
#     print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")
    
    max_cpu, max_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, y, y_masks in train_loader:
            y_hat = model(x, x_masks)                  
#             y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        cpu, ram = print_cpu_usage()
        max_cpu = cpu if cpu > max_cpu else max_cpu
        max_ram = ram if ram > max_ram else max_ram
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()
    print(f"Max CPU usage: {max_cpu:.3f}\tMax RAM % usage: {max_ram}")

In [36]:
def indices_to_multihot(indices, masks, y_hat):
    multihot = torch.zeros_like(y_hat, dtype=torch.float)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx, y_hat.shape[1]).sum(0).float()
    return multihot

In [37]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [38]:
n_epochs = 100
%time train(baseline_rnna, train_loader, val_loader, n_epochs)

CPU: 20.41
RAM %: 59.7
CPU: 20.36
RAM %: 59.7
Epoch: 1 	 Training Loss: 4.323026
Epoch: 1 	 Validation precision@k5: 0.6076, accuracy@k5: 0.3160
Epoch: 1 	 Validation precision@k10: 0.5515, accuracy@k10: 0.4681
Epoch: 1 	 Validation precision@k15: 0.5922, accuracy@k15: 0.5758
Epoch: 1 	 Validation precision@k20: 0.6571, accuracy@k20: 0.6552
Epoch: 1 	 Validation precision@k25: 0.7158, accuracy@k25: 0.7157
Epoch: 1 	 Validation precision@k30: 0.7608, accuracy@k30: 0.7608
CPU: 20.32
RAM %: 59.7
Epoch: 2 	 Training Loss: 3.773407
Epoch: 2 	 Validation precision@k5: 0.6308, accuracy@k5: 0.3300
Epoch: 2 	 Validation precision@k10: 0.5826, accuracy@k10: 0.4951
Epoch: 2 	 Validation precision@k15: 0.6207, accuracy@k15: 0.6037
Epoch: 2 	 Validation precision@k20: 0.6893, accuracy@k20: 0.6874
Epoch: 2 	 Validation precision@k25: 0.7489, accuracy@k25: 0.7488
Epoch: 2 	 Validation precision@k30: 0.7876, accuracy@k30: 0.7876
CPU: 20.33
RAM %: 59.8
Epoch: 3 	 Training Loss: 3.645947
Epoch: 3 	 Vali

CPU: 22.42
RAM %: 60.5
Epoch: 19 	 Training Loss: 3.420774
Epoch: 19 	 Validation precision@k5: 0.6820, accuracy@k5: 0.3602
Epoch: 19 	 Validation precision@k10: 0.6341, accuracy@k10: 0.5414
Epoch: 19 	 Validation precision@k15: 0.6702, accuracy@k15: 0.6521
Epoch: 19 	 Validation precision@k20: 0.7272, accuracy@k20: 0.7251
Epoch: 19 	 Validation precision@k25: 0.7788, accuracy@k25: 0.7786
Epoch: 19 	 Validation precision@k30: 0.8204, accuracy@k30: 0.8204
CPU: 22.54
RAM %: 60.5
Epoch: 20 	 Training Loss: 3.416338
Epoch: 20 	 Validation precision@k5: 0.6890, accuracy@k5: 0.3631
Epoch: 20 	 Validation precision@k10: 0.6351, accuracy@k10: 0.5423
Epoch: 20 	 Validation precision@k15: 0.6702, accuracy@k15: 0.6521
Epoch: 20 	 Validation precision@k20: 0.7268, accuracy@k20: 0.7248
Epoch: 20 	 Validation precision@k25: 0.7775, accuracy@k25: 0.7773
Epoch: 20 	 Validation precision@k30: 0.8202, accuracy@k30: 0.8202
CPU: 22.67
RAM %: 60.5
Epoch: 21 	 Training Loss: 3.413580
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8375, accuracy@k30: 0.8375
CPU: 24.51
RAM %: 60.5
Epoch: 37 	 Training Loss: 3.379836
Epoch: 37 	 Validation precision@k5: 0.7110, accuracy@k5: 0.3751
Epoch: 37 	 Validation precision@k10: 0.6589, accuracy@k10: 0.5612
Epoch: 37 	 Validation precision@k15: 0.6903, accuracy@k15: 0.6711
Epoch: 37 	 Validation precision@k20: 0.7474, accuracy@k20: 0.7453
Epoch: 37 	 Validation precision@k25: 0.7975, accuracy@k25: 0.7973
Epoch: 37 	 Validation precision@k30: 0.8375, accuracy@k30: 0.8375
CPU: 24.65
RAM %: 60.4
Epoch: 38 	 Training Loss: 3.379270
Epoch: 38 	 Validation precision@k5: 0.7181, accuracy@k5: 0.3794
Epoch: 38 	 Validation precision@k10: 0.6585, accuracy@k10: 0.5611
Epoch: 38 	 Validation precision@k15: 0.6904, accuracy@k15: 0.6712
Epoch: 38 	 Validation precision@k20: 0.7472, accuracy@k20: 0.7451
Epoch: 38 	 Validation precision@k25: 0.8007, accuracy@k25: 0.8006
Epoch: 38 	 Validation precision@k30: 0.8389, accuracy@k30: 0.8389
CPU: 24.65
RAM 

Epoch: 54 	 Validation precision@k25: 0.8034, accuracy@k25: 0.8033
Epoch: 54 	 Validation precision@k30: 0.8414, accuracy@k30: 0.8414
CPU: 26.59
RAM %: 60.8
Epoch: 55 	 Training Loss: 3.364645
Epoch: 55 	 Validation precision@k5: 0.7155, accuracy@k5: 0.3769
Epoch: 55 	 Validation precision@k10: 0.6644, accuracy@k10: 0.5652
Epoch: 55 	 Validation precision@k15: 0.6962, accuracy@k15: 0.6767
Epoch: 55 	 Validation precision@k20: 0.7503, accuracy@k20: 0.7482
Epoch: 55 	 Validation precision@k25: 0.8042, accuracy@k25: 0.8040
Epoch: 55 	 Validation precision@k30: 0.8428, accuracy@k30: 0.8428
CPU: 26.68
RAM %: 60.7
Epoch: 56 	 Training Loss: 3.363200
Epoch: 56 	 Validation precision@k5: 0.7194, accuracy@k5: 0.3796
Epoch: 56 	 Validation precision@k10: 0.6649, accuracy@k10: 0.5653
Epoch: 56 	 Validation precision@k15: 0.6964, accuracy@k15: 0.6767
Epoch: 56 	 Validation precision@k20: 0.7514, accuracy@k20: 0.7493
Epoch: 56 	 Validation precision@k25: 0.8040, accuracy@k25: 0.8038
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7504, accuracy@k20: 0.7483
Epoch: 72 	 Validation precision@k25: 0.8046, accuracy@k25: 0.8045
Epoch: 72 	 Validation precision@k30: 0.8450, accuracy@k30: 0.8450
CPU: 28.67
RAM %: 60.8
Epoch: 73 	 Training Loss: 3.357159
Epoch: 73 	 Validation precision@k5: 0.7177, accuracy@k5: 0.3778
Epoch: 73 	 Validation precision@k10: 0.6631, accuracy@k10: 0.5635
Epoch: 73 	 Validation precision@k15: 0.6958, accuracy@k15: 0.6759
Epoch: 73 	 Validation precision@k20: 0.7538, accuracy@k20: 0.7516
Epoch: 73 	 Validation precision@k25: 0.8045, accuracy@k25: 0.8044
Epoch: 73 	 Validation precision@k30: 0.8423, accuracy@k30: 0.8423
CPU: 28.99
RAM %: 60.8
Epoch: 74 	 Training Loss: 3.357522
Epoch: 74 	 Validation precision@k5: 0.7199, accuracy@k5: 0.3800
Epoch: 74 	 Validation precision@k10: 0.6647, accuracy@k10: 0.5654
Epoch: 74 	 Validation precision@k15: 0.6954, accuracy@k15: 0.6760
Epoch: 74 	 Validation precision@k20: 0.7485, accuracy@k20: 0.7464
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.6962, accuracy@k15: 0.6766
Epoch: 90 	 Validation precision@k20: 0.7508, accuracy@k20: 0.7487
Epoch: 90 	 Validation precision@k25: 0.8045, accuracy@k25: 0.8044
Epoch: 90 	 Validation precision@k30: 0.8430, accuracy@k30: 0.8430
CPU: 28.81
RAM %: 60.9
Epoch: 91 	 Training Loss: 3.354171
Epoch: 91 	 Validation precision@k5: 0.7164, accuracy@k5: 0.3783
Epoch: 91 	 Validation precision@k10: 0.6657, accuracy@k10: 0.5658
Epoch: 91 	 Validation precision@k15: 0.6966, accuracy@k15: 0.6767
Epoch: 91 	 Validation precision@k20: 0.7541, accuracy@k20: 0.7521
Epoch: 91 	 Validation precision@k25: 0.8047, accuracy@k25: 0.8046
Epoch: 91 	 Validation precision@k30: 0.8426, accuracy@k30: 0.8426
CPU: 28.75
RAM %: 60.8
Epoch: 92 	 Training Loss: 3.352686
Epoch: 92 	 Validation precision@k5: 0.7179, accuracy@k5: 0.3779
Epoch: 92 	 Validation precision@k10: 0.6650, accuracy@k10: 0.5646
Epoch: 92 	 Validation precision@k15: 0.6939, accuracy@k15: 0.6743
Epoch: 92 	 Val

In [39]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_rnna, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.7390, accuracy@k5: 0.3969
Validation precision@k10: 0.6931, accuracy@k10: 0.5886
Validation precision@k15: 0.7237, accuracy@k15: 0.7001
Validation precision@k20: 0.7733, accuracy@k20: 0.7696
Validation precision@k25: 0.8183, accuracy@k25: 0.8182
Validation precision@k30: 0.8554, accuracy@k30: 0.8554


In [40]:
torch.save(baseline_rnna, os.path.join(CHECKPOINT_PATH, "BaselineRNNa_100.pth"))